# DRLND Project 1 - Navigation (Banana environment)

This notebook presents a solution to the [Udacity Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) Project 1 - Navigation of the [Unity3D](https://unity3d.com/machine-learning/) [Banana environment (Mac version)](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P1/Banana/Banana.app.zip).